In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0..3.0 --quiet

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------------------------- -------- 1.6/2.1 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\absl_py-1.4.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\cachetools-5.3.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\configargparse-1.7-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\google_auth-2.23.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\google_auth_oaut

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\absl_py-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\cachetools-5.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\configargparse-1.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\google_auth-2.23.0-py3.11.egg is deprecat

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\absl_py-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\cachetools-5.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\configargparse-1.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mobeenh20\anaconda3\lib\site-packages\google_auth-2.23.0-py3.11.egg is deprecat

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [4]:
dataset = load_dataset("knkarthick/dialogsum")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/mobeenH20/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Users\mobeenH20\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mobeenH20\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## PEFT:
In modern deep learning, it is possible to pull out number of parameters & see which one of them are trainable parameters.

In [6]:
def trainable_model_parameters(model):
    trainable_model_params = 0
    total_model_params = 0
    for _, param in model.named_parameters():
        total_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable Model Params: {trainable_model_params} total Model Params: {total_model_params} percentage of trainable Params: {trainable_model_params/total_model_params}"

In [7]:
trainable_model_parameters(original_model)

'trainable Model Params: 247577856 total Model Params: 247577856 percentage of trainable Params: 1.0'

## Test Zero Shot Learning

In [8]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following dialogue:
{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors="pt")
output = tokenizer.decode(
         original_model.generate(
             inputs['input_ids'],
             max_new_tokens=200,
         )[0],
    skip_special_tokens = True
)

dash_line = "-".join("" for x in range(0,100))
print(dash_line)
print(f"input Prompt: {prompt}")
print(dash_line)
print(f"baseline human Summary: {summary}")
print(dash_line)
print(f"output summary - Zero Shot: {output}")

---------------------------------------------------------------------------------------------------
input Prompt: 
Summarize the following dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

------------------------------------------------------------------------

### We will going to take the logic of Zero-Shot learning and apply it to all dataset

In [9]:
def tokenize_function(data_instance):
    start_prompt = "Summarize the following dialogue.\n\n"
    end_prompt = "\n\nSummary: "
    
    prompt = [start_prompt + dialogue + end_prompt for dialogue in data_instance['dialogue']]
    
    data_instance['input_ids'] = tokenizer(prompt, padding="max_length", 
                                           truncation=True, return_tensors="pt").input_ids
    data_instance['labels'] = tokenizer(data_instance['summary'], padding="max_length", 
                                           truncation=True, return_tensors="pt").input_ids
    
    return data_instance

In [10]:
# to handle dataset for all batches and splits

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary']) 

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
})

In [12]:
# subsample the dataset for time saving
tokenized_datasets = tokenized_datasets.filter(lambda data_instance, index: index % 100 == 0,  with_indices=True)

print("Dataset: ")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validate: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset: 
Training: (125, 2)
Validate: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


## Fine Tune model with Preprocessed Dataset

In [19]:
output_dir = f"./dialogue-summary-training"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=3,
    max_steps=3
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [20]:
trainer.train()

Step,Training Loss
3,49.166700


TrainOutput(global_step=3, training_loss=49.166666666666664, metrics={'train_runtime': 18.3593, 'train_samples_per_second': 1.307, 'train_steps_per_second': 0.163, 'total_flos': 16434176458752.0, 'train_loss': 49.166666666666664, 'epoch': 0.19})

In [24]:
trainer.save_model(f"./dialogue-summary-training-save")

In [40]:
intruct_model = AutoModelForSeq2SeqLM.from_pretrained("./dialogue-summary-training-save", torch_dtype=torch.float32)

## Evaluation of Model (Human Evaluation)

In [41]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following dialogue:
{dialogue}

Summary:
"""

generation_config = GenerationConfig(max_new_tokens=200, num_beams=1)

inputs = tokenizer(prompt, return_tensors="pt")

original_model_output = tokenizer.decode(original_model.generate(inputs['input_ids'].to('cuda'), generation_config=generation_config)[0], skip_special_tokens = True)
intruct_model_output = tokenizer.decode(intruct_model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens = True)

print(dash_line)
print(f"Baseline Human Summary: {summary}")
print(dash_line)
print(f"Original Model Summary: {original_model_output}")
print(dash_line)
print(f"Instruct Model: {intruct_model_output}")

---------------------------------------------------------------------------------------------------
Baseline Human Summary: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Original Model Summary: #Pork2: You could add a painting program to your software. #Pork2: Is that a bonus? #Pork2: Yes, but I'm not sure what exactly I would need. #Pork2: I'd probably need a faster processor, more powerful hard discs, more memory and a faster modem. #Pork2: I'd probably need a CD-ROM drive too.
---------------------------------------------------------------------------------------------------
Instruct Model: #Person1#: Have you considered upgrading your system? #Person2#: Yes, but I'm not sure what exactly I would need. #Person1#: You could add a painting program to your software. #Person2#: You might also want to upgrade your hardware. #Person1#: You'd probabl

## Evaluation using Rouge (Metric Evaluation)

In [43]:
rouge = evaluate.load('rouge')

In [44]:
dialogues = dataset['test'][0:10]['dialogue']
base_line_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

generation_config = GenerationConfig(max_new_tokens=200, num_beams=1)

for _, dialogue in enumerate(dialogues):
    
    prompt = f"""
Summarize the following dialogue:
{dialogue}

Summary:
"""
    inputs = tokenizer(prompt, return_tensors="pt")

    original_model_output = tokenizer.decode(original_model.generate(inputs['input_ids'].to('cuda'), generation_config=generation_config)[0], skip_special_tokens = True)
    original_model_summaries.append(original_model_output)
    
    intruct_model_output = tokenizer.decode(intruct_model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens = True)
    instruct_model_summaries.append(intruct_model_output)
    
zipped_summaries = list(zip(base_line_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['human_base_line', 'original_model_summaries', 'instruct_model_summaries'])

df
    

,human_base_line,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,Memo to all employees: Urge immediate release ...,The memo is to be distributed to all employees...
1,In order to prevent employees from wasting tim...,The following memo is being distributed to emp...,The memo is to be distributed to all employees...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employee memo on intra-office communication.,The memo is to be distributed to all employees...
3,#Person2# arrives late because of traffic jam....,The traffic in the city is a problem.,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1#: I'm finally here! #Person2#: I got ...,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,The traffic jams in the city are causing a lot...,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are having a separation for 2 m...,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are having a separation for 2 m...,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,"#Person1: Happy Birthday, Brian. #Person2: Hap...",Brian's birthday is coming up.


In [63]:

original_model_results = rouge.compute(
        predictions=original_model_summaries, 
        references=base_line_summaries[0:len(instruct_model_summaries)],
        use_aggregator=True,
        use_stemmer=True
    )

instruct_model_results = rouge.compute(
        predictions=instruct_model_summaries,
        references=base_line_summaries[0:len(instruct_model_summaries)],
        use_aggregator=True,
        use_stemmer=True
    )

print("Original Model: ")
print(original_model_results)

print("Instruct Model: ")
print(instruct_model_results)

Original Model: 
{'rouge1': 0.22731999364485755, 'rouge2': 0.06783724448560786, 'rougeL': 0.1896003008790641, 'rougeLsum': 0.19207685034536015}
Instruct Model: 
{'rouge1': 0.30248625548625546, 'rouge2': 0.10409523809523809, 'rougeL': 0.2452330902330902, 'rougeLsum': 0.2488035838035838}
